# Discussion of the Approach

## Comparison of Models with the same number of Parameters

### What exactly is a “parameter”?

* **Weights**: In a fully-connected (dense) layer, each input–output pair has a weight.
* **Biases**: Each neuron often has an additional bias term.
* **Other learned tensors**: In convolutional layers you learn filter‐kernels; in batch-norm you learn scale and shift parameters; in attention you learn projection matrices; etc.

Whenever the network sees training data, it adjusts these parameters (via backpropagation) to minimize some loss.

### How do you count parameters in common layer types?

1. **Fully-connected (Dense) layer**
   If it maps an input of size *M* to an output of size *N*:

   $$
     \#\text{params} = M \times N \quad(\text{weights})\;+\;N\;(\text{biases})
   $$

2. **Convolutional layer**
   With *C₁* input channels, *C₂* output channels, and a *K×K* kernel:

   $$
     \#\text{params} = C_1 \times C_2 \times K \times K \quad+\;C_2\;(\text{biases})
   $$

3. **Recurrent layer (e.g. LSTM)**
   An LSTM cell mapping *H* hidden units with *I* inputs has roughly:

   $$
     4\bigl((H+I)\times H + H\bigr)
   $$

   (the factor 4 is because of the input, forget, cell, and output gates)

4. **Transformer block**
   A single self‐attention head projects inputs of size *D* to queries, keys, and values, then back again. For *H* heads and a feed-forward dimension *F*:

   $$
     \#\text{params} \approx 3D^2 + D^2 + 2DF + F D + \text{biases}
   $$

   (and you multiply by the number of layers)


### What does the total parameter count tell you?

* **Capacity / expressiveness**
  More parameters allow the network to fit more complex functions—but beyond a point you risk overfitting if you don’t have enough data or regularization.

* **Compute & memory cost**
  Training or inference time and GPU/CPU memory usage both scale roughly with the number of parameters.

* **Data requirement**
  Very large models (hundreds of millions to billions of parameters) generally need large datasets to train effectively.

* **Practical performance**
  While bigger often helps (e.g. scaling laws in language models), diminishing returns set in—double the parameters doesn’t necessarily double accuracy.

### Example comparisons

| Model              | Parameter Count | Notes                             |
| ------------------ | --------------- | --------------------------------- |
| Small MLP          | \~10 Thousand          | Tiny toy model                    |
| ResNet-50 (vision) | \~25 Million          | Mid-sized convolutional network   |
| BERT-Base (NLP)    | \~110 Million         | 12 layers, 12 heads, 768 dim      |
| GPT-3              | 175 Billion           | Very large transformer (few-shot) |


# What kind of baseline does “Fully-Connected” provide?

| Aspect                               | Advantage                                                                                                        | Caveat / why to control for it                                                                                                                                                                                     |
| ------------------------------------ | ---------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Expressive upper bound**           | With *N* hidden units every pair of neurons communicates; universal-approximation is achieved with fewer layers. | Differences you measure may simply reflect *capacity*, not *connectivity*. Control by either (a) equalising **parameter count** across topologies, or (b) giving each topology the same FLOP budget.               |
| **Stable gradient flow**             | Dense skip-paths reduce vanishing/exploding gradients in feed-forward RL policies.                               | In actor-critic methods the critic often over-fits faster in FC nets, destabilising the actor unless you tune entropy or learning-rate schedulers.                                                                 |
| **Acts as a “mainstream” reference** | Reviewers and readers instantly know what FC PPO or FC DQN numbers should look like.                             | It is *not* a free lunch: compute, memory and energy cost scale ≈ O(N²). Sparse or SW graphs often match FC returns at 10–20 % of the multiplications ([arXiv][3], [Proceedings of Machine Learning Research][4]). |

**Recommendation:** report two FC baselines

1. **Same-width FC** (more parameters than your sparse nets) — shows the raw capacity boost.
2. **Equal-parameter FC** (↓width so total weights match the others) — isolates *structural* effects.




# Influence of input / output placement in the graph

* **Path-length to outputs matters.**
  In non-layered graphs you decide which node(s) drive the action logits and which consume observations. If those nodes sit in different modules (in a modular graph) or on opposite sides of a WS ring, you re-introduce long credit-assignment paths even inside a “good” topology.

* **Receptive-field coverage.**
  Inputs wired to a high-degree hub give every hidden neuron fast access to raw state variables; wiring them to a low-degree peripheral node forces hierarchical feature building. Pick one convention (e.g. *all* inputs feed a single “sensor module”; outputs read from a dedicated “motor module”) and keep it fixed across topologies to avoid confounds.

* **Directionality / weight-sharing choices.**
  If you use undirected graphs but directional data-flow (because PyTorch layers run in topological order), you must still assign each edge to “forward” or “skip” positions when you linearise the graph. Different serialisations change effective depth. Log the adjacency *after* topological sorting so the mapping is reproducible.

# Role of the number of hidden nodes (*N*)

1. **Capacity scaling.**
   With sparsity fixed, increasing *N* linearly raises parameter count (∝ N ⟨k⟩). In ER or SBM graphs you can therefore disentangle “more neurons” from “more edges” by holding ⟨k⟩ constant while you sweep *N*.

2. **Topological invariants shift.**
   *Path length* grows ≈ log N in SW graphs but ≈ N¹ in ring-lattices. Doubling *N* can push a graph out of the small-world regime unless you also add shortcuts or raise *k*.

3. **Exploration-to-capacity ratio.**
   RL often operates in the low-data regime relative to parameters. Adding hidden nodes to an FC net hurts sample-efficiency (“square-law” in experience replay size). In sparse nets the same node budget adds fewer weights, so you may see *better* learning curves with larger *N* — a win that pure FC baselines would miss.

4. **Compute envelope.**
   If wall-clock parity is important, remember: dense inference time ∝ N², sparse inference ≈ N ⟨k⟩. On GPUs, small sparse matrices can become *memory-bandwidth* bound; your optimal *N* differs between CPU and GPU clusters.